# Оптимизатор

In [1176]:
import pandas as pd
import numpy as np
import pickle
import json
import import_ipynb
import scipy
from scipy.optimize import minimize
import warnings
from my_libs.calc_features import *
from sklearn.preprocessing import StandardScaler
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 100

In [1177]:
path = "DATA/Optimizer/input.xlsx"

In [1178]:
ls_columns_output = [
    '№ партии',
    '№ плавки',
    'марка стали',
    'диаметр',
    'толщина стенки',
    'Гр. прочн.',
    '1 зона по ВТР закалка',
    '2 зона по ВТР закалка',
    '3 зона по ВТР закалка',
    'шаг балок закалочная печь, сек',
    'Скорость прохождения трубы через спрейер, м/с', 
    't˚ C трубы после спреера',
    '1 зона ВТР и уставка отпуск', 
    '2 зона ВТР и уставка отпуск', 
    '3 зона ВТР и уставка отпуск',
    '4 зона ВТР и уставка отпуск',
    '5 зона ВТР и уставка отпуск',
    'шаг балок отпускная печь, сек',
    'C',
    'Mn',
    'Si',
    'P',
    'S',
    'Cr',
    'Ni',
    'Cu',
    'Al',
    'V',
    'Ti',
    'Nb',
    'Mo',
    'N',
    'B',
    'C-coef',
    'Параметр закалка',
    'Параметр отпуск',
    'Параметр отпуск новый V',
    'Величина зерна',
    'Тип предела текучести (1186)',
    'Дата термообработки',
    'ICD',
    'Примечание'
    ]

In [1179]:
table_for_optimize = pd.read_excel(path, skiprows=1)

## Подготовка полученных от пользователя данных

#### Что будет вводить пользователь?  

Обязательно:
* группа прочности
* марка стали
* диаметр
* толщина стенки
* нижняя и верхняя границы допуска для предела текучести
* нижняя и верхняя границы допуска для предела прочности

Возможно:   
Режим термообработки, который нужно улучшить (он будет начальным приближением для алгоритма оптимизации):

* 1 зона по ВТР закалка
* 2 зона по ВТР закалка
* 3 зона по ВТР закалка
* Скорость прохождения трубы через спрейер, м/с
* 1 зона ВТР и уставка отпуск
* 2 зона ВТР и уставка отпуск
* 3 зона ВТР и уставка отпуск
* 4 зона ВТР и уставка отпуск
* 5 зона ВТР и уставка отпуск
* шаг балок закалочная печь, сек
* шаг балок отпускная печь, сек


Если режим термообработки не вводится, то будет подтягиваться исторический режим с минимальным шагом, в случае если шаг уже 24, режим оптимизироваться не будет (как в случае подтянутого исторического, так и в случае ручного ввода)

Сформируем списки обязательных и необязательных входных данных

In [1180]:
ls_opt_need = [
    'Гр. прочн.',
    'марка стали',
    'толщина стенки',
    'диаметр'
]

bounds = [
    'Предел текучести нижняя граница',
    'Предел текучести верхняя граница',
    'Предел прочности нижняя граница',
    'Предел прочности верхняя граница'
]

ls_additional = [
    '1 зона по ВТР закалка',
    '2 зона по ВТР закалка',
    '3 зона по ВТР закалка',
    'Скорость прохождения трубы через спрейер, м/с',
    '1 зона ВТР и уставка отпуск',
    '2 зона ВТР и уставка отпуск',
    '3 зона ВТР и уставка отпуск',
    '4 зона ВТР и уставка отпуск',
    '5 зона ВТР и уставка отпуск',
    'шаг балок закалочная печь, сек',
    'шаг балок отпускная печь, сек'
]

In [1181]:
table_for_optimize

,Гр. прочн.,марка стали,диаметр,толщина стенки,Предел текучести нижняя граница,Предел текучести верхняя граница,Предел прочности нижняя граница,Предел прочности верхняя граница,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"Скорость прохождения трубы через спрейер, м/с",1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек"
0,К,13ХФА,72,5.5,50.0,NaN,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,К,30Г2,118,12.0,50.0,NaN,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,R95,18ХМФБ,89,9.0,66.8,88.0,70,90.0,840.0,840.0,840.0,0.2,670.0,670.0,657.0,657.0,637.0,28.0,28.0
3,R95,Цу12,89,9.0,77.3,NaN,70,90.0,840.0,840.0,840.0,0.2,670.0,670.0,657.0,657.0,637.0,28.0,28.0


Если верхняя границп не введена, то берем нижнюю + 30

In [1182]:
def fill_up(x):
    if pd.isnull(x['Предел текучести верхняя граница']):
        x['Предел текучести верхняя граница'] = x['Предел текучести нижняя граница']+30
    if pd.isnull(x['Предел прочности верхняя граница']):
        x['Предел прочности верхняя граница'] = x['Предел прочности нижняя граница']+30
    return x
table_for_optimize = table_for_optimize.apply(fill_up, axis=1)

In [1183]:
table_for_optimize['Прочность середина'] = (table_for_optimize[
    'Предел прочности нижняя граница']+table_for_optimize['Предел прочности верхняя граница'])/2.0
table_for_optimize['Текучесть середина'] = (table_for_optimize[
    'Предел текучести нижняя граница']+table_for_optimize['Предел текучести верхняя граница'])/2.0

In [1184]:
table_for_optimize

,Гр. прочн.,марка стали,диаметр,толщина стенки,Предел текучести нижняя граница,Предел текучести верхняя граница,Предел прочности нижняя граница,Предел прочности верхняя граница,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"Скорость прохождения трубы через спрейер, м/с",1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",Прочность середина,Текучесть середина
0,К,13ХФА,72,5.5,50.0,80.0,70,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.0,65.0
1,К,30Г2,118,12.0,50.0,80.0,70,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.0,65.0
2,R95,18ХМФБ,89,9.0,66.8,88.0,70,90.0,840.0,840.0,840.0,0.2,670.0,670.0,657.0,657.0,637.0,28.0,28.0,80.0,77.4
3,R95,Цу12,89,9.0,77.3,107.3,70,90.0,840.0,840.0,840.0,0.2,670.0,670.0,657.0,657.0,637.0,28.0,28.0,80.0,92.3


### Подтягиваем режим, если он не введен

In [1185]:
database[database['марка стали']=='30Г2']['Гр. прочн.'].value_counts().sort_index()

C95          38
E          1099
J55          21
K           746
K72         382
L80тип1     555
M             6
N80типQ    6757
NAN           4
R95         363
Д            22
Л          2502
Name: Гр. прочн., dtype: int64

In [1186]:
database = pd.read_csv('prepared_to_saw_gp_del_bath.csv', index_col=0)

In [1187]:
database = database[~(database['Скорость прохождения трубы через спрейер, м/с']==1)]
database = database[~(pd.isnull(database['Скорость прохождения трубы через спрейер, м/с']))]

#### Очищаем группу прочности, файл пересохранила, теперь это нужно для самого оптимизатора только

In [1188]:
replace_dict_gr = {
    ' ':'',
    '/':'-',
    'ТИП':'тип',
    'К':'K', #русский на английский, везде
    'С':'C',
    'Р':'P',
    'Х':'X',
    'Е':'E',
    'Т':'T',
    'М':'M',
    'У':'Y',
    'Н':'H',
    'В':'B',
    'А':'A',
    'П':'n',
    'О':'O',
    'Т':'T'
}

def fix_h_group(st):
    st = str(st)
    st = st.upper()
    for it in replace_dict_gr:
        st = st.replace(it, replace_dict_gr[it])
    return st

In [1189]:
# database.to_csv('prepared_to_saw_gp_del_bath.csv')

In [1190]:
# database['Гр. прочн.'] = database['Гр. прочн.'].apply(fix_h_group)
# database['Примечание'] = database['Гр. прочн.'].apply(lambda x: None)
# database.to_csv('prepared_to_saw_gp_del_bath.csv')

In [1191]:
# database[ls_columns_output].to_csv('prepared_to_saw_gp_del_bath.csv')

#### Ищем ближайший режим

In [1192]:
def close_value(database, col, value):
    database['diff'] = np.abs(database[col]-value)
    return database.loc[(database['diff']).argmin(),:][col]

In [1193]:
def find_row_close_sort(database, row):
    for col in ls_opt_need:
        tmp = database[database[col]==row[col]]
        if tmp.shape[0]>0:
            database = tmp
        else:
            try:
                value = close_value(database, col, row[col])
                database = database[database[col]==value]
            except TypeError:
                database = database[database[col]==row[col].split('-')[0]]
                if database.shape[0]==0:
                    tmp = row
                    tmp[[col for col in row.index if col not in ls_opt_need]] = None
                    tmp['№ партии'] = 'Error!!! (причина:'+ col+')'
                    return tmp               
    return pd.Series(database[database['Дата термообработки'] == database['Дата термообработки'].max()].iloc[0, :])

In [1194]:
# database['Гр. прочн.'] = database['Гр. прочн.'].apply(fix_h_group)

In [1195]:
# database['Гр. прочн.'].value_counts().index

In [1196]:
def find_close_sort(database, df):
    df['Гр. прочн.'] = df['Гр. прочн.'].apply(fix_h_group)
    df = df.apply(lambda x: find_row_close_sort(database, x), axis=1)
    return df[ls_columns_output]

In [1197]:
answ = find_close_sort(database, table_for_optimize)

#### Оставляем введенные режимы

Немного костыльно, если будет время нкужно будент поменять местами, сначала оставляем те, что уже введены, потом для оставшихся ищем режим.

In [1198]:
try_ = table_for_optimize.copy()
try_ = try_.dropna()

In [1199]:
list(answ.index)
# answ.take([2,3])

[0, 1, 2, 3]

In [1200]:
def get_index_diff(df1,df2):
    return list(set(df1.index).difference(set(df2.index)))

In [1201]:
diff = get_index_diff(answ, try_)

In [1202]:
answ = pd.concat([answ.take(diff), try_])
answ = answ[ls_columns_output]

In [1203]:
answ = pd.concat([answ,table_for_optimize[['Прочность середина', 'Текучесть середина']]], axis=1)

In [1204]:
answ

,№ партии,№ плавки,марка стали,диаметр,толщина стенки,Гр. прочн.,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"шаг балок закалочная печь, сек","Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок отпускная печь, сек",C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,B,C-coef,Параметр закалка,Параметр отпуск,Параметр отпуск новый V,Величина зерна,Тип предела текучести (1186),Дата термообработки,ICD,Примечание,Прочность середина,Текучесть середина
0,412197,z20302,13ХФА,88.90,12.2,K,840.0,840.0,850.0,36.0,0.18,37.5,525.0,525.0,522.0,522.0,521.0,36.0,0.14,0.48,0.24,0.007,0.003,0.53,0.05,0.10,0.033,0.053,0.0017,0.00355,0.007,0.00800,0.00094,0.348000,838.854999,15.844163,11.137841,12.986261,0.2,2014-12-13,0.579200,NaN,85.0,65.0
1,312280,v22228,30Г2,73.02,10.5,K,850.0,850.0,860.0,34.0,0.23,42.5,525.0,580.0,522.0,574.0,521.0,34.0,0.29,1.29,0.23,0.012,0.008,0.06,0.05,0.08,0.030,0.003,0.0040,0.00553,0.005,0.00892,0.00050,0.527267,847.359565,16.143345,10.032797,17.473343,0.2,2013-12-22,1.198637,NaN,85.0,65.0
2,NaN,NaN,18ХМФБ,89.00,9.0,R95,840.0,840.0,840.0,28.0,0.20,NaN,670.0,670.0,657.0,657.0,637.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,77.4
3,NaN,NaN,Цу12,89.00,9.0,R95,840.0,840.0,840.0,28.0,0.20,NaN,670.0,670.0,657.0,657.0,637.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,92.3


### Загрузка моделей для предсказания свойств

In [1205]:
def load_model(dir_name, target):
    model = pickle.load(open(dir_name+'/RF_model_'+target+'.sav', 'rb'))
    ls_need_col = json.load(open(dir_name+'/ls_need_col', "r"))
    try:
        scaler = StandardScaler()
        scale_data = json.load(open(dir_name+'/scaler', "r"))
        scaler.mean_ = scale_data[0]
        scaler.scale_ = scale_data[1]
    except:
        scaler=None
    return model, ls_need_col, scaler

In [1206]:
targets = ['Предел текучести',
           'Врем. сопротивление']

dir_names = ['DATA/MODELS_RF/YS RF valid', 
             'DATA/MODELS_RF/H RF valid']

models_bonds = [
    'Текучесть середина',
    'Прочность середина'
]

In [1222]:
models, ls_need_cols, scalers = [], [], []
for target, dir_name in zip(targets, dir_names):
    model, ls_need_col, scaler = load_model(dir_name, target)
    models.append(model)
    ls_need_cols.append(ls_need_col)
    scalers.append(scaler)

### Подготовка шагов для оптимизатора

Возникла проблема, оптимизатор не работает с двумя моделями, потому что они обучены на разном подмножестве признаков, сейчас пробую обойти это, если не получится, то нужно будет переобучить модельки.

Придумала костыль, живем дальше

In [1223]:
ls_temp_z = [
    '1 зона по ВТР закалка', '2 зона по ВТР закалка',
    '3 зона по ВТР закалка'
]

# ls_temp_o = [
#     '1 зона ВТР и уставка отпуск',
#     '2 зона ВТР и уставка отпуск', '3 зона ВТР и уставка отпуск',
#     '4 зона ВТР и уставка отпуск', '5 зона ВТР и уставка отпуск'
# ]

ls_s_spr = ['Скорость прохождения трубы через спрейер, м/с']

ls_shag = [
    'шаг балок закалочная печь, сек', 'шаг балок отпускная печь, сек'
]

In [1224]:
X_c = answ.copy()
X_c[ls_temp_z] = X_c[ls_temp_z] + 20
X_c[ls_s_spr] = X_c[ls_s_spr] + 0.5
X_c_new = answ.copy()
# X_c[ls_temp_o] = X[ls_temp_o] - 20
X_c_new[ls_shag] = 24

In [1225]:
def uniq(alist):    # Fastest order preserving
    set = {}
    return [set.setdefault(e,e) for e in alist if e not in set]

In [1226]:
ls_need_union = uniq(list(sum(ls_need_cols, [])))

In [1227]:
set(ls_need_cols[0]).symmetric_difference(set(ls_need_cols[1]))

{'3 зона по ВТР закалка', 'Величина зерна', 'Параметр отпуск'}

In [1228]:
X_c_new = X_c_new[ls_need_union+models_bonds].dropna()

In [1229]:
X_c = X_c[ls_need_union+models_bonds].dropna()
X_c

,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",диаметр,толщина стенки,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,Параметр закалка,Параметр отпуск новый V,C-coef,Величина зерна,Параметр отпуск,Текучесть середина,Прочность середина
0,860.0,860.0,870.0,0.68,37.5,525.0,525.0,522.0,522.0,521.0,36.0,36.0,88.90,12.2,0.14,0.48,0.24,0.007,0.003,0.53,0.05,0.10,0.033,0.053,0.0017,0.00355,0.007,0.00800,838.854999,11.137841,0.348000,12.986261,15.844163,65.0,85.0
1,870.0,870.0,880.0,0.73,42.5,525.0,580.0,522.0,574.0,521.0,34.0,34.0,73.02,10.5,0.29,1.29,0.23,0.012,0.008,0.06,0.05,0.08,0.030,0.003,0.0040,0.00553,0.005,0.00892,847.359565,10.032797,0.527267,17.473343,16.143345,65.0,85.0


In [1230]:
a = np.array([2,3,4,6,9])

np.concatenate((a[:2],a[:-3],a[:-3]))

array([2, 3, 2, 3, 2, 3])

In [1239]:
def model_pr(df):
    score = 0
    df_ys = df[:-3]
    df_h = np.concatenate((df[:2],df[3:-4],df[-4:-3]))
    for model, ls_need_col, scaler in zip(models, ls_need_cols, scalers):
        if len(ls_need_col)==31:
            centr = df[-1]
            tmp_score = np.abs(model.predict(df_h.reshape(1, -1)) - centr)
        else:
            centr = df[-2]
            tmp_score = np.abs(model.predict(df_ys.reshape(1, -1)) - centr)
        if tmp_score<2:
            tmp_score=0
        score += tmp_score
    return score

In [1240]:
len(ls_need_cols[0]), len(ls_need_cols[1])

(32, 31)

In [1241]:
X_c_new[ls_need_union+models_bonds]

,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",диаметр,толщина стенки,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,Параметр закалка,Параметр отпуск новый V,C-coef,Величина зерна,Параметр отпуск,Текучесть середина,Прочность середина
0,840.0,840.0,850.0,0.18,37.5,525.0,525.0,522.0,522.0,521.0,24,24,88.90,12.2,0.14,0.48,0.24,0.007,0.003,0.53,0.05,0.10,0.033,0.053,0.0017,0.00355,0.007,0.00800,838.854999,11.137841,0.348000,12.986261,15.844163,65.0,85.0
1,850.0,850.0,860.0,0.23,42.5,525.0,580.0,522.0,574.0,521.0,24,24,73.02,10.5,0.29,1.29,0.23,0.012,0.008,0.06,0.05,0.08,0.030,0.003,0.0040,0.00553,0.005,0.00892,847.359565,10.032797,0.527267,17.473343,16.143345,65.0,85.0


In [1242]:
X_a = X_c_new[ls_need_union+models_bonds].copy()
X_b = X_c[ls_need_union+models_bonds].copy()
for it in range(X_a.shape[0]):
    bounds = [(i, j) for i, j in zip(X_a.iloc[it,:], X_b.iloc[it,:])]
    a = scipy.optimize.differential_evolution(model_pr, bounds)
    print('ВАПРОС a', it, models[1].predict(X_a.loc[it,ls_need_cols[1]].values.reshape(1, -1)),
         models[0].predict(X_a.loc[it,ls_need_cols[0]].values.reshape(1, -1)))
    print('ВАПРОС b', it, models[1].predict(X_b.loc[it,ls_need_cols[1]].values.reshape(1, -1)),
         models[0].predict(X_b.loc[it,ls_need_cols[0]].values.reshape(1, -1)))
    print(a)
    tmp_a_x = a['x']
    df_h = np.concatenate((tmp_a_x[:2],tmp_a_x[3:-4],tmp_a_x[-4:-3]))
    df_ys = tmp_a_x[:-3]
    print('АТВЕТ ', it, models[1].predict(df_h.reshape(1, -1)),
          models[0].predict(df_ys.reshape(1, -1)))

ВАПРОС a 0 [66.77379796] [52.3960344]
ВАПРОС b 0 [66.27776105] [49.86371785]
     fun: 12.033169234988364
 message: 'Optimization terminated successfully.'
    nfev: 8436
     nit: 15
 success: True
       x: array([8.58735335e+02, 8.59922619e+02, 8.64490650e+02, 2.19236345e-01,
       3.75000000e+01, 5.25000000e+02, 5.25000000e+02, 5.22000000e+02,
       5.22000000e+02, 5.21000000e+02, 2.74545144e+01, 2.49807933e+01,
       8.89000000e+01, 1.22000000e+01, 1.40000000e-01, 4.80000000e-01,
       2.40000000e-01, 7.00000000e-03, 3.00000000e-03, 5.30000000e-01,
       5.00000000e-02, 1.00000000e-01, 3.30000000e-02, 5.30000000e-02,
       1.70000000e-03, 3.55000000e-03, 7.00000000e-03, 8.00000000e-03,
       8.38854999e+02, 1.11378411e+01, 3.48000000e-01, 1.29862614e+01,
       1.58441630e+01, 6.50000000e+01, 8.50000000e+01])
АТВЕТ  0 [86.21295238] [52.96683077]
ВАПРОС a 1 [84.73481068] [72.07009996]
ВАПРОС b 1 [86.14216493] [73.11326425]
     fun: 9.013118526167304
 message: 'Optimization 